# Laplace Differential Privacy

By [Armaan Bhojwani](https://armaanb.net) under [Praneeth Vepakomma](https://praneeth.mit.edu/)

This notebook features the following differentially private operations on 1 dimensional dataset of ints with set bounds.
- Laplace Mechanism:
    - Sum
    - Count
    - Mean
    - Histogram
    - Privacy Loss Random Variable
    - Privacy Loss Distribution
    
For operations on a dataset without set bounds (utilizing clipping), see laplace_example_class_height.ipynb
    
### References
- https://programming-dp.com
- B. Pejó and D. Desfontaines, Guide to Differential Privacy Modifications
- https://github.com/google/differential-privacy/blob/main/common_docs/Privacy_Loss_Distributions.pdf

### Status
- Complete

## Parameters

In [1]:
# Privacy
epsilon = 1

# Data
data_len = 150   # Length of dataset
data_low = 0     # Lowest value of dataset
data_high = 99   # Highest value of dataset

## Build the dataset
Create dataset of ints that we can work with

In [2]:
import numpy as np

# Initialize Numpy RNG
rng = np.random.default_rng()

# Increment data_high so that it includes the value specified
data_high += 1

# Create dataset as defined by above parameters
x = rng.integers(low=data_low, high=data_high, size=(data_len))

## Helper functions

In [3]:
import math
from scipy.stats import laplace
from common import *

def laplace_noise(epsilon, sensitivity):
    """ Generate laplace noise given parameters
    Inputs:
        epsilon: epsilon value to use
        sensitivity: sensivitity of the mechanism
    Output:
        laplace noise (float) with specified parameters
    """

    return rng.laplace(scale=sensitivity / epsilon)


def laplace_mech(x, mech, epsilon, sensitivity, verbose=False):
    """ Calculate a differentially private result using laplace noise
    Inputs:
        x: input dataset
        mech: function to run on input, should take single parameter (x)
        epsilon: epsilon value to use
        sensitivity: sensitivity to use
        verbose: print detail
    Output:
        (mech(x), mech(x) + laplace noise)
    """
    mech_x = mech(x)
    noise = laplace_noise(epsilon, sensitivity)

    # We round here so that the result with added noise is still an int, like
    # the input. This is do-able because of DP's post-processing properties
    mech_X = mech_x + round(noise)

    if verbose:
        print(f"Non-private {mech.__name__}: {mech_x}")
        print(f"Private {mech.__name__}:     {mech_X}")

    return (mech_x, mech_X)

## Query implementations
### Sum
Because the data is arbitrary and has publically-known bounds we can manually set the sensitivity to the data's range, however if the upper bound of the data was unknown, we should use a differentially private method of calculating the sensitivity, such as clipping. See the `laplace_example_class_height.ipynb` notebook for an example of this.

In [4]:
sum_sensitivity = data_high - data_low

sum_x, sum_X = laplace_mech(x, np.sum, epsilon, sum_sensitivity, verbose=True)

### Size

The most that the sensitivity could be is 1, because we are doing a count query, and thus when adding or removing a record from the dataset, the most the result can change is 1.

In [5]:
size_sensitivity = 1

size_x, size_X = laplace_mech(x, np.size, epsilon, size_sensitivity, verbose=True)

### Mean
We can build off of the sum and to find the mean. It is important to apply DP to each of the individual steps of finding the mean, as opposed to finding the mean and then applying DP.

In [6]:
# Find non-private mean
mean_x = sum_x / size_x

# Find differentially private mean
mean_X = sum_X / size_X

print(f"Original mean: {mean_x}")

# Round private mean to same number of decimal places as original mean
print(f"Private mean:  {round(mean_X, len(str(mean_x).split('.')[1]))}")

### Differentially private histogram

In [7]:
import matplotlib.pyplot as plt
from matplotlib import ticker

# Parameters
num_bins = 10

# Generate non-private histogram
x_counts, bins = np.histogram(x, bins=num_bins)

# Recount the bins in a private way
X_counts = [laplace_mech(i, lambda x: x, epsilon, 1)[1] for i in x_counts]


def plot_hist(bins, weights):
    """ Styles DP histogram
    Inputs:
        bins: array of bin boundaries to use
        weights: counts for each bin
    Output:
        Matplotlib plot ready to be plotted. Add whatever extra styling you 
        want, then call plt.show().
    """
    
    ax = plt.gca()

    # Set Y-Axis to reasonable bounds
    if min(weights) > 20:
        ax.set_ylim([0.9 * min(weights), max(weights) + 0.1 * min(weights)])
    else:
        ax.set_ylim([0, max(weights) + 2])
        ax.set_yticks(
            [i for i in range(int(max(weights)) + 2) if (i % 2 == 0)])

    # Set axis ticks
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    plt.xticks(bins)

    # Add vertical lines between bars
    [plt.axvline(x=i, color="w") for i in bins]

    # Add counts to each bar
    centers = [(bins[i] + bins[i + 1]) / 2 for i in range(np.size(weights))]
    for yc, xc in zip(weights, centers):
        ax.text(xc, yc + 0.5, "%d" % yc, ha="center")

    # Plot
    plt.hist(bins[:-1], bins, weights=weights)


# Plot non-private histogram
plt.title("Non-private histogram")
plot_hist(bins, x_counts)
plt.show()

# Plot private histogram
plt.title("Private histogram")
plot_hist(bins, X_counts)
plt.show()

## Quantifying privacy loss

### Calculating privacy loss random variable (PLRV)

In [8]:
from tqdm.notebook import tqdm


def calc_PLRV(x, mech, epsilon, sensitivity, num_samples=1, verbose=False):
    """ Calculates the privacy loss random variable of a laplace DP mechanism
    Inputs:
        x: dataset to operate on
        mech: query to apply on x
        epsilon: epsilon value to use
        sensitivity: mechanism sensitivity
        num_samples: how many time to sample to PLRV
        verbose: print detail
    Output:
        an array of samples of the PLRV with length num_samples
    """
    
    # Calculate original and private mech(x)
    if verbose: print("On original database:")
    mech_x, mech_X = laplace_mech(x,
                                  mech,
                                  epsilon,
                                  sensitivity,
                                  verbose=verbose)

    output = []

    for i in tqdm(range(num_samples), disable=(num_samples < 5000)):
        # Calculate original and private mech(x) on neighbouring dataset
        x2 = create_neighbour(x, verbose=verbose)
        if verbose: print("On neighbouring database:")
        mech_x2, mech_X2 = laplace_mech(x2,
                                        mech,
                                        epsilon,
                                        sensitivity,
                                        verbose=verbose)

        # Calculate PLRV
        # See section 3.1 of Google paper
        delta = abs(mech_x - mech_x2)
        delta_tilde = delta / (sensitivity / epsilon)

        w = laplace.rvs(0, 1)
        if w <= 0:
            output.append(delta_tilde)
        elif w >= delta_tilde:
            output.append(-delta_tilde)
        elif 0 < w and w < delta_tilde:
            output.append(delta_tilde - 2 * w)

    return output


size_plrv = calc_PLRV(x, np.size, epsilon, size_sensitivity, 1, verbose=True)
print(f"The PLRV of a size query is: {size_plrv}")

### Plotting privacy loss distribution (PLD)

In [9]:
# Parameters
num_samples = 20000
num_bins = 50

def plot_PLD(*plrv_func_args):
    """ Plots the PLD
    Inputs:
        plrv_func_args: arguments to pass to calc_PLRV function
    Output:
        Matplotlib plots and prints
    """
    data = calc_PLRV(*plrv_func_args)
    plt.hist(data, bins=num_bins)
    plt.title("Privacy loss distribution histogram for query")
    plt.show()

    plt.hist(data, bins=500, cumulative=True, histtype='step')
    plt.title("Privacy loss distribution CDF for query")
    plt.show()

    abs_data = [abs(i) for i in data]
    expected = np.mean(abs_data)
    print(f"The expected absolute value of the PLRV is {expected}")

### PLD of size query

In [10]:
plot_PLD(x, np.size, epsilon, size_sensitivity, num_samples)